# 머신 러닝 교과서 - 파이토치편

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/ml-with-pytorch/blob/main/ch15/ch15_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## 패키지 버전 체크

check_packages.py 스크립트에서 로드하기 위해 폴더를 추가합니다:

In [1]:
import sys

# 코랩의 경우 깃허브 저장소로부터 python_environment_check.py를 다운로드 합니다.
if 'google.colab' in sys.modules:
    !wget https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/python_environment_check.py
    !wget https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/1268-0.txt
else:
    sys.path.insert(0, '..')

--2023-09-05 08:00:39--  https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/python_environment_check.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1629 (1.6K) [text/plain]
Saving to: ‘python_environment_check.py’

python_environment_ 100%[===================>]   1.59K  --.-KB/s    in 0s      

2023-09-05 08:00:39 (28.6 MB/s) - ‘python_environment_check.py’ saved [1629/1629]

--2023-09-05 08:00:40--  https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/1268-0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response..

권장 패키지 버전을 확인하세요:

In [1]:
from python_environment_check import check_packages


d = {
    'torch': '1.8.0',
}
check_packages(d)

[OK] Your Python version is 3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]


c:\venv39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[OK] torch 1.8.1+cpu


# 15장 - 순환 신경망으로 순차 데이터 모델링 (파트 3/3)

**목차**

- 파이토치로 시퀀스 모델링을 위한 RNN 구현
  - 두 번째 프로젝트: 텐서플로로 글자 단위 언어 모델 구현
    - 데이터셋 전처리
    - 문자 수준의 RNN 모델 만들기
    - 평가 단계: 새로운 텍스트 생성
- 요약

In [3]:
%pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ------------- -------------------------- 2.6/7.8 MB 12.6 MB/s eta 0:00:01
   -------------------------------------- - 7.6/7.8 MB 20.4 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 19.3 MB/s  0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 27.4 MB/s  0:00:00

   ----------- ---------------------------- 2/7 [importlib-resources]
   ----------------- ---------------------- 3/7 [fonttools]
   ----------------- ---------------------- 3/7 [fonttools]
   ----------------- ---------------------- 3/7 [fonttools]
   ----------------- ---------------------- 3/7 [fonttools]
   ----------------- ---------------------- 3/7 [fonttools]
   ----------------- ---------------------- 3/7 [fonttools]
   ---------

In [4]:
import matplotlib
from IPython.display import Image
%matplotlib inline

## 15.3.2 두 번째 프로젝트: 텐서플로로 글자 단위 언어 모델 구현

In [4]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_11.png', width=500)

### 데이터셋 전처리

1874년 출간한 신비의 섬(The Mysterious Island)

In [5]:
import numpy as np

## 텍스트 읽고 전처리하기
with open('1268-0.txt', 'r', encoding="utf8") as fp:
    text=fp.read()

start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')

text = text[start_indx:end_indx]
char_set = set(text)
print('전체 길이:', len(text))
print('고유한 문자:', len(char_set))

전체 길이: 1112350
고유한 문자: 80


In [7]:
text[:50]


'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by Anthony '

In [6]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_12.png', width=500)

In [8]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i,ch in enumerate(chars_sorted)} # 문자 → 정수 인덱스 딕셔너리를 생성
char_array = np.array(chars_sorted)

text_encoded = np.array( # 각 문자를 char2int를 이용해 정수 인덱스로 변환
    [char2int[ch] for ch in text],
    dtype=np.int32)

print('인코딩된 텍스트 크기: ', text_encoded.shape)

print(text[:15], '     == 인코딩 ==> ', text_encoded[:15])
print(text_encoded[15:21], ' == 디코딩  ==> ', ''.join(char_array[text_encoded[15:21]]))

인코딩된 텍스트 크기:  (1112350,)
THE MYSTERIOUS       == 인코딩 ==>  [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]
[33 43 36 25 38 28]  == 디코딩  ==>  ISLAND


In [9]:
for ex in text_encoded[:5]:
    print('{} -> {}'.format(ex, char_array[ex]))

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


In [9]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_13.png', width=500)

파이토치 텍스트 생성 모델:

- sequence 길이 = 40

 > 시퀀스 길이가 길면 더 의미있는 문장 생성

 > short 시퀀스:  문맥 무시하고 개별 단어를 정확히 감지

 > 적절한 길이의 시퀀스 길이 찾는 것: 하이퍼파라미터 최적화 문제

In [10]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_14.png', width=500)

입력 시퀀스: 처음 40개 문자

target 시퀀스:1개 밀린 다음 40개 문자

 X = [c0, c1, c2, ..., c39]
 Y = [c1, c2, c3, ..., c40]

각 시점마다 다음 문자를 예측

 -> RNN에서 흔히 쓰는 many-to-many + teacher forcing

In [10]:
seq_length = 40
chunk_size = seq_length + 1 # 41개 문자

text_chunks = [text_encoded[i:i+chunk_size]
               for i in range(len(text_encoded)-chunk_size+1)]
# text_encoded 전체를 한 글자씩 이동(sliding) 하면서 길이 41짜리 조각을 전부 생성
## 조사:
for seq in text_chunks[:1]: # 전체 데이터 중 첫 번째 chunk 하나만 확인 디버깅 / 이해용
    input_seq = seq[:seq_length] # chunk 앞 40개 문자 모델의 입력 X
    target = seq[seq_length] # chunk 앞 40개 문자 모델의 입력 X
    print(input_seq, ' -> ', target)
    print(repr(''.join(char_array[input_seq])),
          ' -> ', repr(''.join(char_array[target])))

[44 32 29  1 37 48 43 44 29 42 33 39 45 43  1 33 43 36 25 38 28  1  6  6
  6  0  0  0  0  0 40 67 64 53 70 52 54 53  1 51]  ->  74
'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'  ->  'y'


현재 코드 (many-to-one)

 - 출력은 마지막 1개, loss 1개, 구현이 단순

__getitem__()은 파이썬 객체가 “인덱싱 대상”처럼 보일 때 자동으로 호출되는 핵심 메커니즘

1) obj[key]

 > obj.__getitem__(key)



In [ ]:
class A:
    def __getitem__(self, idx):
        print("called", idx)
        return idx * 10

a = A()
a[3]


2) obj[start:stop:step] # 슬라이싱

 > obj.__getitem__(slice(start, stop, step))



3) for x in obj:
    ...

 > iter(obj) 시도 → __iter__()

 > 없으면 __getitem__(0), __getitem__(1), ...

In [ ]:
class B:
    def __getitem__(self, idx):
        if idx >= 3:
            raise IndexError
        return idx

for x in B():
    print(x)


4) x in obj

 > __contains__()가 없으면

 > 반복 시도 → __getitem__() 사용

In [ ]:
class C:
    def __getitem__(self, idx):
        if idx == 0: return 'a'
        if idx == 1: return 'b'
        raise IndexError

'a' in C()   # True


5) PyTorch Dataset

  dataset[i]

  > dataset.__getitem__(i)



In [ ]:
import torch
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, text_chunks):
        self.text_chunks = text_chunks

    def __len__(self):
        return len(self.text_chunks)

    def __getitem__(self, idx):
        text_chunk = self.text_chunks[idx]
        return text_chunk[:-1].long(), text_chunk[1:].long() # 입력 시퀀스, target 시퀀스를 반환

seq_dataset = TextDataset(torch.tensor(text_chunks))

enumerate(seq_dataset)

 > enumerate는 인덱스 번호를 붙여주는 래퍼(wrapper)일 뿐 데이터 접근 방식에는 관여하지 않는다

In [ ]:
# enumerate() 하는 일
count = 0
for element in iterable: # seq_dataset가 iterable 객체이므로 in에서 __getitem__() 호출
    yield count, element
    count += 1


In [12]:
for i, (seq, target) in enumerate(seq_dataset): # TextDataset.__getitem__()이 자동으로 호출
    print('입력 (x):', repr(''.join(char_array[seq])))
    print('타깃 (y):', repr(''.join(char_array[target])))
    print()
    if i == 1:
        break

입력 (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
타깃 (y): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'

입력 (x): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'
타깃 (y): 'E MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by '



In [13]:
device = torch.device("cuda:0")
# device = 'cpu'

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64

torch.manual_seed(1)
seq_dl = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
# 미니 배치로 변환

### 문자 수준의 RNN 모델 만들기

In [16]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)


CUDA available: False
PyTorch version: 1.8.1+cpu
CUDA version: None


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [18]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size,
                           batch_first=True)
        self.fc = nn.Linear(rnn_hidden_size, vocab_size)

    def forward(self, x, hidden, cell):
        out = self.embedding(x).unsqueeze(1)
        out, (hidden, cell) = self.rnn(out, (hidden, cell))
        out = self.fc(out).reshape(out.size(0), -1)
        return out, hidden, cell

    def init_hidden(self, batch_size):
        hidden = torch.zeros(1, batch_size, self.rnn_hidden_size)
        cell = torch.zeros(1, batch_size, self.rnn_hidden_size)
        return hidden.to(device), cell.to(device)

vocab_size = len(char_array)
embed_dim = 256
rnn_hidden_size = 512

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size)
model = model.to(device)
model

RNN(
  (embedding): Embedding(80, 256)
  (rnn): LSTM(256, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=80, bias=True)
)

In [ ]:
# 시간 소요 30분 이상 걸림
# loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

num_epochs = 10000

torch.manual_seed(1)

for epoch in range(num_epochs):
    hidden, cell = model.init_hidden(batch_size)
    seq_batch, target_batch = next(iter(seq_dl))
    seq_batch = seq_batch.to(device)
    target_batch = target_batch.to(device)
    optimizer.zero_grad()
    loss = 0
    for c in range(seq_length):
        pred, hidden, cell = model(seq_batch[:, c], hidden, cell)
        loss += loss_fn(pred, target_batch[:, c])
    loss.backward()
    optimizer.step()
    loss = loss.item()/seq_length
    if epoch % 500 == 0:
        print(f'에포크 {epoch} 손실: {loss:.4f}')

에포크 0 손실: 4.3706
에포크 500 손실: 1.4209
에포크 1000 손실: 1.2304
에포크 1500 손실: 1.1878
에포크 2000 손실: 1.2030
에포크 2500 손실: 1.1323
에포크 3000 손실: 1.2019
에포크 3500 손실: 1.1798
에포크 4000 손실: 1.1201


KeyboardInterrupt: 

### 평가 단계: 새로운 텍스트 생성

softmax(xi​)= ​exi​​ /∑​exj

Categorical(logits=z)는 내부적으로 softmax(z)를 적용하여 확률 분포를 만들므로,

Categorical(probs=softmax(z))와 같은 확률 분포를 정의한다.

In [ ]:
from torch.distributions.categorical import Categorical

torch.manual_seed(1)

logits = torch.tensor([[1.0, 1.0, 1.0]]) # 상대적인인 크기 > [1.0,1.0,2.0] 시도하기 
# 확률로 바꾸기 전의 원시 점수(raw score)
print('확률:', nn.functional.softmax(logits, dim=1).numpy()[0])

m = Categorical(logits=logits) # Categorical(probs=softmax(logits))
samples = m.sample((10,))# 확률에 비례한 다양성
print(samples.numpy())

확률: [0.33333334 0.33333334 0.33333334]
[[0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [2]
 [1]
 [1]]


In [23]:
torch.manual_seed(1)

logits = torch.tensor([[1.0, 1.0, 3.0]])

print('확률:', nn.functional.softmax(logits, dim=1).numpy()[0])

m = Categorical(logits=logits)
samples = m.sample((10,))

print(samples.numpy())

확률: [0.10650698 0.10650698 0.78698605]
[[0]
 [2]
 [2]
 [1]
 [2]
 [1]
 [2]
 [2]
 [2]
 [2]]


로짓(logits)은 모델이 각 클래스를 얼마나 선호하는지를 나타내는 원시 점수이며,

이 점수에 softmax를 적용하면 확률이 된다.

언어 모델의 기본 가정 (가장 중요)

 - 문자 언어 모델은 다음을 학습.

𝑃(𝑐𝑡∣𝑐1,𝑐2,...,𝑐𝑡−1)

 > “지금까지 본 문자들이 주어졌을 때, 다음 문자가 무엇일까?”

(지금까지의 문맥) + (현재 입력)
            ↓
      다음 토큰 예측


In [24]:
# 문자 단위 RNN/LSTM 언어 모델로 텍스트를 생성(sample)
def sample(model, starting_str, # 모델, 생성의 시작 문자열
           len_generated_text=500, # 새로 생성할 문자 수
           scale_factor=1.0): # 로짓 스케일링

    encoded_input = torch.tensor([char2int[s] for s in starting_str]) # 시작 문자열의 각 문자를 정수 인덱스로 변환
    encoded_input = torch.reshape(encoded_input, (1, -1)) # (1, len(starting_str))

    generated_str = starting_str

    model.eval()
    hidden, cell = model.init_hidden(1) # RNN/LSTM의 초기 은닉 상태
    hidden = hidden.to('cpu')
    cell = cell.to('cpu')
    for c in range(len(starting_str)-1): #시작 문자열의 마지막 글자 전까지 
        _, hidden, cell = model(encoded_input[:, c].view(1), hidden, cell) # 한 글자씩 모델에 입력 출력 버리고, hidden, cell만 갱신

    last_char = encoded_input[:, -1] # 마지막 문자를 따로 저장
    for i in range(len_generated_text): # 새로 생성할 문자 수 만큼 반복
        logits, hidden, cell = model(last_char.view(1), hidden, cell) # 지금까지 나온 마지막 문자”를 입력으로 주어
        ## 그 다음에 나올 문자”의 확률을 예측하기 위해서
        logits = torch.squeeze(logits, 0)
        scaled_logits = logits * scale_factor
        m = Categorical(logits=scaled_logits) # 로짓 → softmax → 확률 분포
        # 다음 문자를 뽑기 위한 확률 모델
        last_char = m.sample()
        generated_str += str(char_array[last_char])

    return generated_str

torch.manual_seed(1)
model.to('cpu')
print(sample(model, starting_str='The island'))

The island
had been observed thus. But in this iron had reflections, animals of a fire, staircourse you will suppose to think was stranger to wards sedions in this side, and a larbood hands. It is on
salt towards the morning, and was abundant,
as if they had done molding, thereford diefs penchorathy, Herbert.

“Euctric,” answered Pencroft and his life could have been troubled him, and Pencroft was not been to the works.

He sailed at the corners of a fancalic columns of Sucoe. “But what will be long in th


모든 생성형 AI(GPT 포함)의 가장 핵심 원리: 

- 3가지 생성형 AI의 필수 조건을 모두 만족:

1) 확률 모델

  > m = Categorical(logits=scaled_logits)

    last_char = m.sample()

2) 자기 출력(self-output)을 다시 입력으로 사용한다

 > last_char → model → next_char → 다시 last_char

   >> 모델이 스스로 생성한 결과를 다음 입력으로 사용

   >> 외부 정답 사용않는다 ⇒ “자기 주도적 생성”

3) 열린 길이(open-ended generation)

   for i in range(len_generated_text):


Autoregressive Generative Model (자기회귀 생성 모델)

생성형 AI의 정의:

1) 세계(언어)를 확률 분포로 학습

2) 그 분포에서 자율적으로 샘플

3) 샘플 결과를 다시 입력

4) 의미 있는 시퀀스를 만들어냄

이 코드는 단순한 RNN 예제가 아니라,

확률·자기회귀·샘플링이라는 생성형 AI의 핵심 원리를

가장 작고 명확한 형태로 구현한 ‘미니 GPT’.

* **예측 가능성 vs. 무작위성**

In [21]:
logits = torch.tensor([[1.0, 1.0, 3.0]])

print('스케일 조정 전의 확률:        ', nn.functional.softmax(logits, dim=1).numpy()[0])

print('0.5배 조정 후 확률:', nn.functional.softmax(0.5*logits, dim=1).numpy()[0])

print('0.1배 조정 후 확률:', nn.functional.softmax(0.1*logits, dim=1).numpy()[0])

스케일 조정 전의 확률:         [0.10650698 0.10650698 0.78698605]
0.5배 조정 후 확률: [0.21194156 0.21194156 0.57611686]
0.1배 조정 후 확률: [0.3104238  0.3104238  0.37915248]


In [22]:
torch.manual_seed(1)
print(sample(model, starting_str='The island',
             scale_factor=2.0))

The island was an
insure, and the sand would be seen their close of the house, and the intelligence and a straight of the surrounded on the shores of the island would have an abundant which would have been able to followed by the power.

“No, my friend,” said Herbert, “and as if you, Pencroft and Herbert was that the first rivales were completed the operation of the colonists and his companions.

Herbert, who was seen the colonists’ pottering of the coast of the colonists were honest for the convicts, the


In [23]:
torch.manual_seed(1)
print(sample(model, starting_str='The island',
             scale_factor=0.5))

The island
would bozent jemutterinuana rollum. “Lazzing ISLCalson of anihmic; Cape, I pier
‘pinutnely,” Tide,” said Coolleguark: their arras; t miys by--nisaries,” he akyed Cyrtam,
smple;
to hain; in--nder, Cor exhaoial;, an inhago was deed--a hruck frosts, Jacriam clvinomes’, at lo,-koednoated, grar, he escen;
Positm ton ”ickally
severally drawn I am-Neb “or here, pierced how ly histe’s privatebk, we diok recemevin,
leanging?” sighted, two
CPratch fallewled! Yon.

After an
weakle nearl fifly Lur, will we


# 요약